In [9]:
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [16]:
# cifar 100 데이터 로드
(x_train,y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data()

# train을 초기에 26000으로 가져옴 ( 이후 또 분할 )
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=26000, random_state=5)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=5)
x_test, x_not, y_test, y_not = train_test_split(x_test, y_test, test_size=0.4, random_state=5)

# Squeeze와 one hot을 사용하여 dense 층을 25로
y_train = tf.squeeze(tf.one_hot(y_train, 25))
y_valid = tf.squeeze(tf.one_hot(y_valid, 25))
y_test = tf.squeeze(tf.one_hot(y_test, 25))

print('train data')
print(x_train.shape)
print(y_train.shape)

print('test data')
print(x_test.shape)
print(y_test.shape)

print('valid data')
print(x_valid.shape)
print(y_valid.shape)

train data
(19200, 32, 32, 3)
(19200, 25)
test data
(6000, 32, 32, 3)
(6000, 25)
valid data
(4800, 32, 32, 3)
(4800, 25)


In [1]:
x_resize_train = []
x_resize_test = []
x_resize_valid = []
for idx in range(len(x_train)):
  temp = x_train[idx]
  temp = cv2.resize(temp, (224,224))
  x_resize_train.append(temp)
x_resize_train = np.array(x_resize_train)
x_train = x_resize_train

for idx in range(len(x_test)):
  temp = x_test[idx]
  temp = cv2.resize(temp, (224,224))
  x_resize_test.append(temp)
x_resize_test = np.array(x_resize_test)
x_test = x_resize_test

for idx in range(len(x_valid)):
  temp = x_valid[idx]
  temp = cv2.resize(temp, (224,224))
  x_resize_valid.append(temp)
x_resize_valid = np.array(x_resize_valid)
x_valid = x_resize_valid

print('resized data shape')
print(x_train[0].shape)
print(x_test[0].shape)
print(x_valid[0].shape)

NameError: ignored

In [13]:
base_model = tf.keras.applications.ResNet50(weights='imagenet', input_shape=(224, 224, 3))
base_model = tf.keras.models.Model(base_model.inputs, base_model.layers[-2].output)
x = base_model.output
pred = tf.keras.layers.Dense(25, activation='softmax')(x)
model = tf.keras.models.Model(inputs=base_model.input , outputs=pred)

opt = tf.keras.optimizers.Adam(learning_rate=0.00001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])

model.summary()

102973440/102967424 [==============================] - 1s 0us/step
Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________

In [14]:
for layer in model.layers:
  layer.trainable = True

for idx, layer in enumerate(model.layers):
  if idx < 100:
    layer.trainable = False

model.summary()
history = model.fit(x=x_train, y=y_train, batch_size=32, epochs=5, validation_data=(x_valid, y_valid))

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [15]:
results = model.evaluate(x_test, y_test, batch_size=32)

print('test accuracy')
print(results[1])

188/188 [==============================] - 19s 99ms/step - loss: 1.1994 - acc: 0.6678
test accuracy
0.6678333282470703
